# **Exploratory Data Analysis Project**

In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# load the dataset
dataset = pd.read_csv('dataset.csv')
dataset.head()

,STATISTIC,Statistic Label,TLIST(Q1),Quarter,C02199V02655,Sex,C04283V05060,Education Attainment Level,C02076V02508,Age Group,UNIT,VALUE
0,QLF50C01,Employment rate,20191,2019Q1,-,Both sexes,-,Levels of Education (Levels 0-8),-,All ages,%,74.9
1,QLF50C01,Employment rate,20191,2019Q1,-,Both sexes,-,Levels of Education (Levels 0-8),365,20 - 24 years,%,64.0
2,QLF50C01,Employment rate,20191,2019Q1,-,Both sexes,-,Levels of Education (Levels 0-8),410,25 - 29 years,%,80.1
3,QLF50C01,Employment rate,20191,2019Q1,-,Both sexes,-,Levels of Education (Levels 0-8),4251,25 - 54 years,%,80.1
4,QLF50C01,Employment rate,20191,2019Q1,-,Both sexes,-,Levels of Education (Levels 0-8),440,30 - 34 years,%,81.4
